<a href="https://colab.research.google.com/github/george-zakharov/ml_notebooks/blob/master/Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 6

Датасет: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

#### Light: 

решить задачу классификации на предложенном датасете с использованием Boosting моделей.



#### Pro: 

- нормализовать, стандартизовать датасет и применить техники работы с не сбалансированным датасетом;

- решить задачу классификации на предложенном датасете с использованием Boosting моделей..

# Load Libs

In [0]:
import pandas as pd

# Load dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Loading dataset
wine = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/6/data/winequality-red.csv')

In [0]:
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [0]:
# Check class balance
wine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64